In [1]:
import tensorflow as tf
import numpy as np

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
IMG_PXL_SIZE = 50
HM_SLICES = 20

In [ ]:
n_classes = 2

x = tf.placeholder('float')
y = tf.placeholder('float')

In [5]:
keep_rate = 0.8
keep_prob = tf.placeholder(tf.float32)

In [ ]:
def conv3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1,1,1,1,1], padding='SAME')

def maxpool3d(x):
    #                        size of window         movement of window
    return tf.nn.max_pool3d(x, ksize=[1,2,2,2,1], strides=[1,2,2,2,1], padding='SAME')

In [ ]:
def convolutional_neural_network(x):
    weights = {'W_conv1':tf.Variable(tf.random_normal([5,5,5,1,32])),
               'W_conv2':tf.Variable(tf.random_normal([5,5,5,32,64])),
               'W_fc':tf.Variable(tf.random_normal([54080,1024])),
               'out':tf.Variable(tf.random_normal([1024, n_classes]))}

    biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
               'b_conv2':tf.Variable(tf.random_normal([64])),
               'b_fc':tf.Variable(tf.random_normal([1024])),
               'out':tf.Variable(tf.random_normal([n_classes]))}

    x = tf.reshape(x, shape=[-1, IMG_PXL_SIZE, IMG_PXL_SIZE, HM_SLICES, 1])

    conv1 = tf.nn.relu(conv3d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool3d(conv1)

    conv2 = tf.nn.relu(conv3d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool3d(conv2)
    

    fc = tf.reshape(conv2,[-1, 54080])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc = tf.nn.dropout(fc, keep_rate)

    output = tf.matmul(fc, weights['out'])+biases['out']

    return output

## Training Neural Netwrok

I tried training over the complete train set but my system broke down due to the heavy processing so I used chunks of data to train, the results are definitely not accurate because this model has not been trained on complete data but still it works pretty good.

** My system is a Core i7 with 12gigs of RAM and NVIDIA 1050Ti**

In [4]:
def train_neural_network(x):

    train_data = np.load('traindata-50-50-20.npy')
    test_data = np.load('testdata-50-50-20.npy')
    
    train = train_data[:10]
    test = train_data[10:12]

    prediction = convolutional_neural_network(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)

    hm_epochs = 5
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            success_total = 0
            attempt_total = 0
            for data in train:
                attempt_total += 1
                try:

                    X = data[0]
                    Y = data[1]
                    _, c = sess.run([optimizer, cost], feed_dict={x: X, y: Y})
                    epoch_loss += c
                    success_total += 1

                except Exception as e:
                    print('Error occured')

            print('Epoch', epoch+1, 'completed out of',hm_epochs,'loss:',epoch_loss, 'success_rate:', success_total/attempt_total)
            
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x: [i[0] for i in test], y: [i[1] for i in test]}))

train_neural_network(x)

NameError: name 'X' is not defined